В этом задании будет использоваться датасет digits из sklearn.datasets. Оставьте последние 25% объектов для контроля качества, разделив X и y на X_train, y_train и X_test, y_test.

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами 1NN с RandomForestClassifier из sklearn на 1000 деревьях.

Задание 1

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка N log N сравнений (строже говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в KNeighborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных ball tree и kd tree.

Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1.

Задание 2

Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

Как отправить
Когда работа будет готова, вы можете загрузить файлы для каждой части задания на вкладке 'Мои работы'.

In [20]:
from sklearn import datasets,  tree, metrics, ensemble, linear_model, model_selection, neighbors

import numpy as np
import pandas as pd


In [3]:
digits = datasets.load_digits()
X = digits.data
y= digits.target

In [36]:
p = 0.75

idx = int(p * X.shape[0]) + 1

X_train, X_test = np.split(X, [idx])
y_train, y_test = np.split(y, [idx])

In [12]:

rf_clf = ensemble.RandomForestClassifier(n_estimators=1000)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
f_err_rate = 1 - metrics.accuracy_score(y_test, y_pred_rf)
print (f_err_rate)

0.06681514476614703


In [13]:
with open('3_2.txt','w') as f:
    f.write(str(f_err_rate))

In [14]:
def dist(x,y):
    return np.sqrt(np.sum(y-x) ** 2)

In [38]:
one_clf = neighbors.KNeighborsClassifier(n_neighbors = 1)
#one_clf = KNeighborsClassifier(n_neighbors=1, weights='distance')
#one_clf = KNeighborsClassifier(n_neighbors=1, weights='uniform')
one_clf.fit(X_train, y_train)

one_predicted = one_clf.predict(X_test)
accuracy = metrics.accuracy_score(y_test, one_predicted)
ans = 1 - accuracy

In [39]:
with open('3_1.txt','w') as f:
    f.write(str(ans))

In [37]:
y_pred_knn = []
for test_value in X_test:
    ind_min_metric = 0
    min_metric = dist(test_value, X_train[0])
    
    for index, train_value in enumerate(X_train):
        metric = dist(test_value, train_value)
        if metric < min_metric:
            min_metric = metric
            ind_min_metric = index
            
    y_pred_knn.append(y_train[ind_min_metric])

TypeError: 'numpy.float64' object is not callable

In [32]:
knn_err_rate = 1 - metrics.accuracy_score(y_test, y_pred_knn)
knn_err_rate

0.8841870824053453